In [1]:
import PyLog as pl
import PyGol as pygol
from itertools import chain
from texttable import Texttable

In [2]:
BK_1 = ["starboard(X,Y) :- sector(X,Y,S), >=(S,1), =<(S,15)",
"port(X,Y)      :- sector(X,Y,S), >=(S,17), =<(S,31)",
"forward(X,Y)   :- sector(X,Y,S), >=(S,0), =<(S,7)",
"forward(X,Y)   :- sector(X,Y,S), >=(S,25), =<(S,31)",
"aft(X,Y)       :- sector(X,Y,S), >=(S,9), =<(S,23)", 
"ahead(X,Y) :- sector(X,Y,S), ==(S,0)",
    "bow(X,Y) :- sector(X,Y,S), >=(S,1), =<(S,4)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,4)",
    "forward_beam(X,Y) :- sector(X,Y,S), >=(S,5), =<(S,7)",
    "beam(X,Y) :- sector(X,Y,S), ==(S,8)",
    "aft_beam(X,Y) :- sector(X,Y,S), >=(S,9), =<(S,11)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,12)",
    "quarter(X,Y) :- sector(X,Y,S), >=(S,12), =<(S,15)",
    "astern(X,Y) :- sector(X,Y,S), ==(S,16)",
    "quarter(X,Y) :- sector(X,Y,S), >=(S,17), =<(S,20)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,20)",
    "aft_beam(X,Y) :- sector(X,Y,S), >=(S,21), =<(S,23)",
    "beam(X,Y) :- sector(X,Y,S), ==(S,24)",
    "forward_beam(X,Y) :- sector(X,Y,S), >=(S,25), =<(S,27)",
    "broad(X,Y) :- sector(X,Y,S), ==(S,28)",
    "bow(X,Y) :- sector(X,Y,S), >=(S,28), =<(S,31)",
 
]

In [3]:
scenario = ["risk_collision(agent_p1,other_p1)", "sector(other_p1,agent_p1,22)", 
            "risk_collision(agent_p2,other_p2)","sector(other_p2,agent_p2,12)",
            "risk_collision(agent_n1,other_n1)", "sector(other_n1,agent_n1,23)",
            "risk_collision(agent_n2,other_n2)","sector(other_n2,agent_n2,23)",
            "sector(other_n3,agent_n3,12)",
            ]

In [4]:
example_pos = ["applies(agent_p1,other_p1, rule_13_overtaking)", 
               "applies(agent_p2,other_p2, rule_13_overtaking)"
                ]
example_neg = ["applies(agent_n1,other_n1, rule_13_overtaking)",
               "applies(agent_n2,other_n2, rule_13_overtaking)",
               "applies(agent_n3,other_n3, rule_13_overtaking)",
               ]

In [5]:

kb_mt_2 = pl.KnowledgeBase()
kb_mt_2.add_clauses(BK_1)
P1, N1 = pygol.relevant_literals(file=scenario, constant_set = [],  container = "memory",
                                positive_example=example_pos,
                                negative_example= example_neg, tqdm_disable=True)  
P2 = pl.modify_bcrl(P1, BK_1, [])
N2 = pl.modify_bcrl(N1, BK_1, [])
#print(P2)
finak_bk_1 = list(chain.from_iterable(P2.values()))
finak_bk_2 = list(chain.from_iterable(N2.values()))


finak_bk = finak_bk_1+finak_bk_2

P, N = pygol.bottom_clause_generation(file=finak_bk,  
                                       constant_set = ["rule_13_overtaking"],  
                                       container = "memory",
                                positive_example=example_pos, 
                                negative_example=example_neg, constrains=[],tqdm_disable=True)

for i,j in P.items():
    print(f"{i} - {j}")
for i,j in N.items():
    print(f"{i} - {j}")

P3,N3 = pygol.numerical_learning(P,N, mode = {"sector":2}, midpoint=False, pos_learning=True)



Train_P = {i:j for i,j in P3.items()}
Train_N = {i:j for i,j in N3.items()}
#print(Train_P)
#print(Train_N)
model_1 = pygol.pygol_learn(Train_P, Train_N,  constant_set = ["rule_13_overtaking"],  max_literals=3,  exact_literals=True, 
                    key_size=len(Train_P), min_pos=1, distinct=True, eval_metrics=False, verbose=True)
print("Learned Model: ")
print(model_1.hypothesis)
print(model_1.time_learn)

applies(agent_p1,other_p1, rule_13_overtaking) - ['risk_collision(A,B)', 'sector(B,A,22)', 'port(B,A)', 'aft(B,A)', 'aft_beam(B,A)']
applies(agent_p2,other_p2, rule_13_overtaking) - ['risk_collision(A,B)', 'sector(B,A,12)', 'starboard(B,A)', 'aft(B,A)', 'broad(B,A)', 'quarter(B,A)']
applies(agent_n1,other_n1, rule_13_overtaking) - ['risk_collision(A,B)', 'sector(B,A,23)', 'port(B,A)', 'aft(B,A)', 'aft_beam(B,A)']
applies(agent_n2,other_n2, rule_13_overtaking) - ['risk_collision(A,B)', 'sector(B,A,23)', 'port(B,A)', 'aft(B,A)', 'aft_beam(B,A)']
applies(agent_n3,other_n3, rule_13_overtaking) - ['sector(B,A,12)', 'starboard(B,A)', 'aft(B,A)', 'broad(B,A)', 'quarter(B,A)']
Positive Example0
	 applies(A,B,rule_13_overtaking):-aft_beam(B,A)&sector(B,A,C)&risk_collision(A,B)
	 applies(A,B,rule_13_overtaking):-risk_collision(A,B)&sector(B,A,C)&range(C,12,22)
	 applies(A,B,rule_13_overtaking):-port(B,A)&sector(B,A,C)&risk_collision(A,B)
	 applies(A,B,rule_13_overtaking):-aft_beam(B,A)&port(B,A)